In [1]:
import pandas as pd
import os
from scipy.stats import spearmanr, pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import r2_score
import statsmodels.api as sm
import plotly.express as px
import numpy as np

In [2]:
## Read in abstracts_final.csv into new dataframe called df_abstracts
df_abstracts = pd.read_csv('abstracts_final.csv')

## Exclude year 2024 since we don't have NIH data for that year
df_abstracts_subset = df_abstracts[df_abstracts['year'] != 2024]

## Rename smart_institution5 as institution
df_abstracts_subset.rename(columns = {'smart_institution5': 'institution'}, inplace = True)

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_59000/562399838.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstracts_subset.rename(columns = {'smart_institution5': 'institution'}, inplace = True)


## Obtain NIH Funding Data
We obtained NIH Funding Data from Blue Ridge from 2016 to 2023

In [3]:
## Read in all excel files in the nih_funding directory
## combine into a single dataframe and add a column for year

## Set the directory
directory = 'nih_funding'

## Get a list of all the files in the directory
files = os.listdir(directory)

## Create an empty list to store the dataframes
dfs = []

## Loop through the files
for file in files:
    ## Check if the file is an excel file
    if file.endswith('.xlsx') or file.endswith('.xls'):
        ## Read the file into a dataframe
        df = pd.read_excel(os.path.join(directory, file))

        ## Remove the first few rows
        df = df[1:]

        ## Set column names to ['rank', 'institution', 'funding']
        df.columns = ['rank', 'institution', 'funding']

        # df.columns = df.iloc[0]

        ## Add a column for the year
        df['year'] = file.split(' ')[-1].replace('.xlsx', '').replace('.xls', '')
        ## Append the dataframe to the list

        dfs.append(df)
        
## Concatenate the dataframes
df = pd.concat(dfs)

In [4]:
## Clean the original dataframe to make it more usable

def clean_data(df):
    ## Turn rank into an int, if na set to 0
    df['rank'] = pd.to_numeric(df['rank'], errors='coerce').fillna(0).astype(int)

    ## Remove rank 0
    df = df[df['rank'] > 0]

    return df

df_funding = clean_data(df)

## Sort df_funding by year and rank
df_funding.sort_values(['year', 'rank'], ascending=[True, True], inplace=True)

## Change institution to title case
df_funding['institution'] = df_funding['institution'].str.title()

df_funding

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_59000/2190675179.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_funding.sort_values(['year', 'rank'], ascending=[True, True], inplace=True)
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_59000/2190675179.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_funding['institution'] = df_funding['institution'].str.title()


,rank,institution,funding,year
1,1,University Of Pennsylvania,19621244,2016
2,2,Washington University,19280840,2016
3,3,Duke University,16546762,2016
4,4,University Of Michigan,16531254,2016
5,5,University Of Pittsburgh At Pittsburgh,15586617,2016
...,...,...,...,...
73,73,University Of Mississippi Med Ctr,268394,2023
74,74,Temple University,220778,2023
75,75,Suny Upstate Medical Center,203750,2023
76,76,University Of Massachusetts Med Sch Worcester,163058,2023


In [5]:
## Create a new dataframe called df_funding_subset
## include only top 10 institutions per year
df_funding_subset = df_funding[df_funding['rank'] <= 10]
df_funding_subset

,rank,institution,funding,year
1,1,University Of Pennsylvania,19621244,2016
2,2,Washington University,19280840,2016
3,3,Duke University,16546762,2016
4,4,University Of Michigan,16531254,2016
5,5,University Of Pittsburgh At Pittsburgh,15586617,2016
...,...,...,...,...
6,6,University Of Pennsylvania,17861753,2023
7,7,Stanford University,17735229,2023
8,8,University Of Pittsburgh,17394455,2023
9,9,University Of California San Francisco,17307614,2023


In [6]:
## Plot the top 10 institutions for each year in a bar graph
## Use plotly express
## Each year should be a facet

import plotly.express as px

fig = px.bar(
    df_funding_subset,
    x='institution',
    y='funding',
    color='year',
    facet_col='year',
    facet_col_wrap=4,
    text='institution')

## Don't need X axes to match
fig.update_xaxes(matches=None)

## Remove x tick labels
fig.update_xaxes(showticklabels=False)

## Make pretty
## Make 1920 x 1080
## Make sure all labels show up nicely

fig.update_layout(
    autosize=False,
    width=1920,
    height=1080,
    title='Top 10 NIH Funded Institutions by Year',
    yaxis_title='Funding',
    xaxis_title='Institution',
    font_family = 'Inter',
)


fig.show()


In [7]:
df_abstracts_institution_year = df_abstracts_subset.groupby(['year', 'institution']).agg(
    count = ('control_number', 'nunique')
)

df_abstracts_institution_year.reset_index(inplace=True)

## institution_year by year and institution, later years first
df_abstracts_institution_year.sort_values(['count', 'institution', 'year'], ascending=[False, False, False], inplace=True)
df_abstracts_institution_year.to_csv('abstracts_institution_year.csv')

I created a map to match the institution name in the Blue Ridge Report to what we have in abstracts

In [8]:
## import nih_institution_map into df_institution_map
df_institution_map = pd.read_csv('nih_institution_map_updated.csv')
df_institution_map

,old_name,new_name
0,University Of Pennsylvania,Hospital Of The University Of Pennsylvania
1,Duke University,Duke University Medical Center
2,University Of Pittsburgh At Pittsburgh,University Of Pittsburgh
3,"University Of California, San Francisco",University Of California - San Francisco
4,University Of Wisconsin-Madison,University Of Wisconsin
...,...,...
87,University Of Tennessee Hlth Sci Ctr,University Of Tennessee Health Science Center
88,Suny Upstate Medical Center,Suny Upstate Medical University
89,Stony Brook University Medical Center,University Of Massachusetts Medical School
90,Brown University,Brown University School Of Medicine


In [9]:
## Use institution map to replace institution in df_funding if institution is not in old_name
## Create a new column called new_name

df_funding_renamed = df_funding.merge(df_institution_map, left_on='institution', right_on='old_name', how='left')
# df_funding_renamed['institution_clean'] = df_funding_renamed['new_name'].fillna(df_funding['institution'])
df_funding_renamed

,rank,institution,funding,year,old_name,new_name
0,1,University Of Pennsylvania,19621244,2016,University Of Pennsylvania,Hospital Of The University Of Pennsylvania
1,2,Washington University,19280840,2016,Washington University,Washington University In St. Louis
2,3,Duke University,16546762,2016,Duke University,Duke University Medical Center
3,4,University Of Michigan,16531254,2016,NaN,NaN
4,5,University Of Pittsburgh At Pittsburgh,15586617,2016,University Of Pittsburgh At Pittsburgh,University Of Pittsburgh
...,...,...,...,...,...,...
605,73,University Of Mississippi Med Ctr,268394,2023,University Of Mississippi Med Ctr,University Of Mississippi
606,74,Temple University,220778,2023,NaN,NaN
607,75,Suny Upstate Medical Center,203750,2023,Suny Upstate Medical Center,Suny Upstate Medical University
608,76,University Of Massachusetts Med Sch Worcester,163058,2023,University Of Massachusetts Med Sch Worcester,University Of Massachusetts Medical School


In [10]:
## If new_name is not null, use new_name, otherwise use institution
df_funding_renamed['institution_clean'] = df_funding_renamed['new_name'].fillna(df_funding_renamed['institution'])

## Get rid of columns we don't need
df_funding_final = df_funding_renamed[['rank', 'funding', 'year', 'institution_clean']]

## rename institution_clean to institution
df_funding_final.rename(columns={'institution_clean': 'institution'}, inplace=True)

df_funding_final['year'] = df_funding_final['year'].astype(int)

df_funding_final

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_59000/1578134069.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_59000/1578134069.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,rank,funding,year,institution
0,1,19621244,2016,Hospital Of The University Of Pennsylvania
1,2,19280840,2016,Washington University In St. Louis
2,3,16546762,2016,Duke University Medical Center
3,4,16531254,2016,University Of Michigan
4,5,15586617,2016,University Of Pittsburgh
...,...,...,...,...
605,73,268394,2023,University Of Mississippi
606,74,220778,2023,Temple University
607,75,203750,2023,Suny Upstate Medical University
608,76,163058,2023,University Of Massachusetts Medical School


In [11]:
## Merge in abstract data
df_merge_by_year = df_funding_final.merge(df_abstracts_institution_year, on=['institution', 'year'], how='left')
df_merge_by_year

## Fill in missing values with 0
df_merge_by_year['count'] = df_merge_by_year['count'].fillna(0)

# df_merge_by_year.to_csv('merge_by_year.csv')

In [12]:
# Calculate spearmanr correlation by year
correlations_list = []
for year, group in df_merge_by_year.groupby('year'):
    correlation, p_value = spearmanr(group['funding'], group['count'])
    correlations_list.append([year, correlation, p_value])

# Creating a df_merge_by_year dataframe from the list
correlations_df_corrected = pd.DataFrame(correlations_list, columns=['Year', 'Correlation', 'P_Value'])

correlations_df_corrected

,Year,Correlation,P_Value
0,2016,0.451344,4.274601e-05
1,2017,0.532693,7.296328e-07
2,2018,0.494084,4.956143e-06
3,2019,0.444701,6.410049e-05
4,2020,0.517669,1.426389e-06
5,2021,0.474273,1.504741e-05
6,2022,0.520749,1.418942e-06
7,2023,0.505725,2.711483e-06


In [13]:
## Create a dataframe that groups df_merge_by_year by institution and provides the total number of abstracts and avg NIH funding
df_merge_by_institution = df_merge_by_year.groupby('institution').agg(
    count = ('count', 'sum'),
    funding = ('funding', 'mean')
)

df_merge_by_institution.dropna(inplace=True)
df_merge_by_institution.reset_index(inplace=True)
df_merge_by_institution.sort_values(['funding', 'count'], ascending=[False, False], inplace=True)

df_merge_by_institution['funding'] = df_merge_by_institution['funding'].astype(float)

In [14]:
## Create a log transform of NIH Funding

df_merge_by_institution['log_average_funding'] = np.log1p(df_merge_by_institution['funding'])
df_merge_by_institution

df_merge_by_institution

,institution,count,funding,log_average_funding
11,Duke University Medical Center,118.0,2.873265e+07,17.173545
65,University Of Michigan,373.0,2.407716e+07,16.996774
93,Washington University In St. Louis,51.0,2.321553e+07,16.960332
75,University Of Pittsburgh,160.0,1.850336e+07,16.733463
17,Hospital Of The University Of Pennsylvania,228.0,1.415646e+07,16.465682
...,...,...,...,...
67,University Of Mississippi,8.0,2.296840e+05,12.344464
39,Temple University,16.0,2.207780e+05,12.304917
77,University Of South Alabama,8.0,1.835900e+05,12.120466
63,University Of Massachusetts Worcester,0.0,1.645590e+05,12.011031


In [15]:
# Calculate spearmanr correlation overall
correlations_list = []
correlation, p_value = spearmanr(df_merge_by_institution['funding'], df_merge_by_institution['count'])

correlation, p_value

(0.6540039427710439, 5.015828644235987e-13)

## Create a Linear Regression 
We want to compare actual abstracts from predicated abstracts

## Linear Regression (untransformed)

In [16]:
# Prepare data for linear regression model
X = df_merge_by_institution[['funding']]  # Independent variable
y = df_merge_by_institution['count']  # Dependent variable

# Fit a linear regression model
model = LinearRegression()

# Linear only transformation
## model.fit(X, y)

model.fit(X, y)

# Predict abstract counts based on NIH funding
y_pred = model.predict(X)

# Calculating the R-squared value
r2 = r2_score(y, y_pred)

# Coefficients
coefficients = model.coef_

# Intercept
intercept = model.intercept_

## Print all the values
r2, coefficients, intercept

(0.306628976616373, array([8.37053907e-06]), 26.079870601461387)

## Linear Regression (Log Transform NIH Funding)

In [17]:
# Prepare the data for regression analysis
X_log = df_merge_by_institution['log_average_funding'].values.reshape(-1, 1)  # Independent variable
y = df_merge_by_institution['count'].values  # Dependent variable

# Fit the linear regression model
model = LinearRegression()
model.fit(X_log, y)

# Predict the total abstracts based on log-transformed average NIH funding
y_pred = model.predict(X_log)

# Calculating the R-squared value
r2 = r2_score(y, y_pred)

# Coefficients
coefficients = model.coef_

# Intercept
intercept = model.intercept_

## Print all the values
r2, coefficients, intercept

(0.3191499815560267, array([36.44807757]), -468.4964490229051)

## Poisson Regression

In [18]:
# # Prepare the data for regression analysis
# X_log_glm = df_merge_by_institution['log_average_funding'].values.reshape(-1, 1)  # Independent variable
# y_glm = df_merge_by_institution['count'].values  # Dependent variable

# # Fit the linear regression model
# poisson_glm = PoissonRegressor(alpha=0, max_iter=1000)  # alpha=0 implies no regularization
# poisson_glm.fit(X_log_glm, y_glm)

# # Predict the total abstracts based on log-transformed average NIH funding
# y_pred_glm = poisson_glm.predict(X_log_glm)

# # Calculate the residuals
# residuals = y_glm - y_pred_glm

# # Calculating the R-squared value
# r2 = r2_score(y_glm, y_pred_glm)

# # Coefficients
# coefficients = poisson_glm.coef_

# # Intercept
# intercept = poisson_glm.intercept_

# ## Print all the values
# r2, coefficients, intercept

In [21]:
# Calculate residuals (difference between actual and predicted abstract counts)
df_merge_by_institution['residuals'] = y - y_pred

# # Calculate residuals (difference between actual and predicted abstract counts)
# df_merge_by_institution['residuals_glm'] = y_glm - y_pred_glm

df_merge_by_institution

,institution,count,funding,log_average_funding,residuals
11,Duke University Medical Center,118.0,2.873265e+07,17.173545,-39.446239
65,University Of Michigan,373.0,2.407716e+07,16.996774,221.996701
93,Washington University In St. Louis,51.0,2.321553e+07,16.960332,-98.675048
75,University Of Pittsburgh,160.0,1.850336e+07,16.733463,18.593893
17,Hospital Of The University Of Pennsylvania,228.0,1.415646e+07,16.465682,96.354011
...,...,...,...,...,...
67,University Of Mississippi,8.0,2.296840e+05,12.344464,26.564465
39,Temple University,16.0,2.207780e+05,12.304917,36.005862
77,University Of South Alabama,8.0,1.835900e+05,12.120466,34.728774
63,University Of Massachusetts Worcester,0.0,1.645590e+05,12.011031,30.717477


In [22]:

# # Identify overperformers and underperformers based on residuals_glm
# overperformers = df_merge_by_institution[df_merge_by_institution['residuals_glm'] > 0]  # Actual count is higher than predicted
# underperformers = df_merge_by_institution[df_merge_by_institution['residuals_glm'] < 0]  # Actual count is lower than predicted

# # Recalculate residuals_glm for all (in case of previous filtering)
# df_merge_by_institution['predicted_count'] = poisson_glm.predict(X_log_glm)
# df_merge_by_institution['residuals_glm'] = df_merge_by_institution['count'] - df_merge_by_institution['predicted_count']

# ## Turn residuals_glm into percentiles from 0 to 100
# df_merge_by_institution['residual_percentile'] = df_merge_by_institution['residuals_glm'].rank(pct=True) * 100

# ## Calculate the median
# df_merge_by_institution['residual_percentile'].median()

# df_merge_by_institution

In [23]:

# Identify overperformers and underperformers based on residuals
overperformers = df_merge_by_institution[df_merge_by_institution['residuals'] > 0]  # Actual count is higher than predicted
underperformers = df_merge_by_institution[df_merge_by_institution['residuals'] < 0]  # Actual count is lower than predicted

# Recalculate residuals for all (in case of previous filtering)
df_merge_by_institution['predicted_count'] = model.predict(X_log)
df_merge_by_institution['residuals'] = df_merge_by_institution['count'] - df_merge_by_institution['predicted_count']

## Turn residuals into percentiles from 0 to 100
df_merge_by_institution['residual_percentile'] = df_merge_by_institution['residuals'].rank(pct=True) * 100

## Calculate the median
df_merge_by_institution['residual_percentile'].median()

df_merge_by_institution

,institution,count,funding,log_average_funding,residuals,predicted_count,residual_percentile
11,Duke University Medical Center,118.0,2.873265e+07,17.173545,-39.446239,157.446239,27.083333
65,University Of Michigan,373.0,2.407716e+07,16.996774,221.996701,151.003299,98.958333
93,Washington University In St. Louis,51.0,2.321553e+07,16.960332,-98.675048,149.675048,3.125000
75,University Of Pittsburgh,160.0,1.850336e+07,16.733463,18.593893,141.406107,67.708333
17,Hospital Of The University Of Pennsylvania,228.0,1.415646e+07,16.465682,96.354011,131.645989,96.875000
...,...,...,...,...,...,...,...
67,University Of Mississippi,8.0,2.296840e+05,12.344464,26.564465,-18.564465,70.833333
39,Temple University,16.0,2.207780e+05,12.304917,36.005862,-20.005862,75.000000
77,University Of South Alabama,8.0,1.835900e+05,12.120466,34.728774,-26.728774,73.958333
63,University Of Massachusetts Worcester,0.0,1.645590e+05,12.011031,30.717477,-30.717477,71.875000


In [24]:
## Create a correlation plot between 'Surgery' and 'count'
## use plotly express

fig = px.scatter(
    df_merge_by_institution,
    x='log_average_funding',
    y='count',
    color='residual_percentile',
    color_continuous_scale='RdBu',
    color_continuous_midpoint=df_merge_by_institution['residual_percentile'].median(),
    hover_name='institution',
    hover_data=['funding', 'count', 'residuals'],
    title="NIH Funding vs. Abstract Count: Overperformers and Underperformers",
    labels={'funding': 'NIH Funding ($)', 'count': 'Abstract Count (2016-2023)'}
)

# Increase marker size
fig.update_traces(marker=dict(size=12))

fig.add_traces(px.line(df_merge_by_institution, 
                       x='log_average_funding', 
                       y='predicted_count',
                       labels={'x': 'funding', 
                               'y': 'Predicted Abstract Count'}).data)

## Change line color to black and thickness to 6
fig.update_traces(line=dict(color='black', width=6))

fig.update_layout(
    width=1600,
    height=720,
    font=dict(
        family="Inter",
        size=16)
)

## Remove colorbar
fig.update_layout(coloraxis_showscale=False)

fig.show()

fig.write_image('funding_vs_abstracts.svg')


In [25]:
# Prepare the independent variables, adding a constant term for the intercept
X_log = df_merge_by_institution[['log_average_funding']]
X_log = sm.add_constant(X)  # Adds a constant term to the predictor
y = df_merge_by_institution['count']

# Fit the linear regression model
model = sm.OLS(y, X_log).fit()

# Obtain the model summary
summary = model.summary()

# Print the summary
print(summary)

                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     41.57
Date:                Tue, 21 May 2024   Prob (F-statistic):           4.86e-09
Time:                        16:15:51   Log-Likelihood:                -542.55
No. Observations:                  96   AIC:                             1089.
Df Residuals:                      94   BIC:                             1094.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.0799      9.207      2.833      0.0

In [26]:
overperformers.sort_values('residuals', ascending=False).head(10)

,institution,count,funding,log_average_funding,residuals
44,University Of Alabama At Birmingham,512.0,6.706132e+06,15.718533,407.586139
65,University Of Michigan,373.0,2.407716e+07,16.996774,221.996701
3,Baylor College Of Medicine,196.0,4.357511e+06,15.287412,107.299680
17,Hospital Of The University Of Pennsylvania,228.0,1.415646e+07,16.465682,96.354011
23,Medical College Of Wisconsin,164.0,2.550944e+06,14.751974,94.815348
20,Johns Hopkins University School Of Medicine,208.0,9.640581e+06,16.081492,90.356978
95,Yale University School Of Medicine,171.0,3.926777e+06,15.183330,86.093268
84,University Of Texas Southwestern Medical Center,156.0,3.092595e+06,14.944522,79.797370
45,University Of Arizona,142.0,2.121198e+06,14.567492,79.539367
48,University Of California - Los Angeles,193.0,9.345914e+06,16.050450,76.488407
